In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta


def coletar_dados(page, data_inicial, data_final, situacao):
    url = "https://api.com/dados"
    params = {
        "_page": page,
        "data_inicial": data_inicial.strftime('%d-%m-%Y'),
        "data_final": data_final.strftime('%d-%m-%Y'),
        "situacao": situacao
    }
    resposta = requests.get(url, params=params)
    return resposta


limite_por_pagina = 10
tempo_limite_minutos = 15
requisicoes_feitas = 0
tempo_inicio = time.time()


intervalo_dias = 180
hoje = datetime.now().date()


data_nova_inicial = hoje - timedelta(days=60)
data_nova_final = hoje + timedelta(days=60)


data_em_andamento_inicial = hoje - timedelta(days=90)
data_em_andamento_final = hoje - timedelta(days=30)

data_finalizada_inicial = hoje - timedelta(days=180)
data_finalizada_final = hoje - timedelta(days=91)


configuracoes_consulta = [
    {"situacao": "nova", "data_inicial": data_nova_inicial, "data_final": data_nova_final},
    {"situacao": "em_andamento", "data_inicial": data_em_andamento_inicial, "data_final": data_em_andamento_final},
    {"situacao": "finalizada", "data_inicial": data_finalizada_inicial, "data_final": data_finalizada_final}
]


for config in configuracoes_consulta:
    situacao = config["situacao"]
    data_inicial = config["data_inicial"]
    data_final = config["data_final"]
    
    page = 1
    nova_tabela = pd.DataFrame()
    
    while True:

        if requisicoes_feitas >= limite_por_pagina:
            tempo_decorrido = time.time() - tempo_inicio
            if tempo_decorrido < (tempo_limite_minutos * 60):
                tempo_espera = (tempo_limite_minutos * 60) - tempo_decorrido
                print(f"Esperando {tempo_espera:.1f} segundos para continuar...")
                time.sleep(tempo_espera)
            requisicoes_feitas = 0
            tempo_inicio = time.time()
        
        resposta = coletar_dados(page, data_inicial, data_final, situacao)
        
        if resposta.status_code != 200:
            print(f"Erro ao acessar a API: {resposta.status_code}")
            break
        
        informacoes = resposta.json()
        if not informacoes:
            break
        
        tabela = pd.DataFrame(informacoes)
        nova_tabela = pd.concat([nova_tabela, tabela], ignore_index=True)
        page += 1
        requisicoes_feitas += 1
    
    excel_file = f"dados_coletados_{situacao}.xlsx"
    nova_tabela.to_excel(excel_file, index=False)
    
    print(f"Dados da situacao '{situacao}' salvos em: {excel_file}")

print("Processo concluído.")


In [ ]:
import pandas as pd
import requests
import time
import os
from datetime import datetime, timedelta
import logging

def coletar_dados(pagina, data_inicial, data_final, situacao):
    url = "https://teste/rest/views/consultar/394af454-3b96-11ee-a683-dwdawdwdawdw"
    headers = {
        "chave-api": "dawdwaawd-54554-11ef-aa2b-dadwawwadwa"
    }
    params = {
        "pagina": pagina,
        "data_inicial": data_inicial.strftime('%d-%m-%Y'),
        "data_final": data_final.strftime('%d-%m-%Y'),
        "situacao": situacao
    }
    resposta = requests.get(url, headers=headers, params=params)
    return resposta

# Variáveis de controle de tempo
limite_por_pagina = 8
tempo_limite_minutos = 16
requisicoes_feitas = 0
tempo_inicio = time.time()

# Data atual
hoje = datetime.now().date()

# Definindo intervalos de datas
data_intervalo = 60
datas_config = [
    {"nome": "nova", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "em_andamento", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "finalizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "aguardando", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "realizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)}
]

# Define o diretório base e cria o diretório para hoje
diretorio_base = r"\\diretorio\algun"
diretorio_hoje = os.path.join(diretorio_base, hoje.strftime('%d-%m-%Y'))

# Cria o diretório se não existir
if not os.path.exists(diretorio_hoje):
    os.makedirs(diretorio_hoje)

# Configuração do logging
log_file_path = os.path.join(diretorio_hoje, "log.txt")
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

# Processamento das configurações
for config in datas_config:
    situacao = config["nome"]
    data_inicial = config["data_inicial"]
    data_final = config["data_final"]
    
    pagina = 1
    nova_tabela = pd.DataFrame()
    
    while True:
        # Verifica se já atingiu o limite de requisições por página
        if requisicoes_feitas >= limite_por_pagina:
            tempo_decorrido = time.time() - tempo_inicio
            if tempo_decorrido < (tempo_limite_minutos * 60):
                tempo_espera = (tempo_limite_minutos * 60) - tempo_decorrido
                logging.info(f"Esperando {tempo_espera:.1f} segundos para continuar...")
                time.sleep(tempo_espera)
            requisicoes_feitas = 0
            tempo_inicio = time.time()
        
        try:
            resposta = coletar_dados(pagina, data_inicial, data_final, situacao)
        except requests.RequestException as e:
            logging.error(f"Erro ao fazer a requisição: {e}")
            break

        # Captura de logs usando logging
        if resposta.status_code == 200:
            logging.info(f"Requisição bem-sucedida: {resposta.url}")
        else:
            logging.error(f"Erro ao acessar a API: {resposta.status_code} - {resposta.text}")
            break
        
        try:
            informacoes = resposta.json()
        except ValueError as e:
            logging.error(f"Erro ao decodificar JSON: {e}")
            break

        if not informacoes:
            logging.info(f"Nenhum dado recebido na página {pagina} para a situação '{situacao}'.")
            break
        
        tabela = pd.DataFrame(informacoes)
        nova_tabela = pd.concat([nova_tabela, tabela], ignore_index=True)
        pagina += 1
        requisicoes_feitas += 1

    # Salvamento dos dados coletados em Excel
    excel_file = os.path.join(diretorio_hoje, f"dados_coletados_{situacao}.xlsx")
    nova_tabela.to_excel(excel_file, index=False)
    
    logging.info(f"Dados da situação '{situacao}' salvos em: {excel_file}")

print("Processo concluído.")

In [ ]:
import pandas as pd
import requests
import time
import os
from datetime import datetime, timedelta
import logging

def coletar_dados(pagina, data_inicial, data_final, situacao):
    url = "https://teste/rest/views/consultar/3455435-3b96-11ee-a683-dwdawdwdawdw"
    headers = {
        "chave-api": "dawdwaawd-32423-11ef-aa2b-dadwawwadwa"
    }
    params = {
        "pagina": pagina,
        "data_inicial": data_inicial.strftime('%d-%m-%Y'),
        "data_final": data_final.strftime('%d-%m-%Y'),
        "situacao": situacao
    }
    resposta = requests.get(url, headers=headers, params=params)
    return resposta

limite_diferentes_consultas = 10  
tempo_limite_minutos = 15  

requisicoes_diferentes_consultas = 0
tempo_inicio = time.time()

hoje = datetime.now().date()


data_intervalo = 60
datas_config = [
    {"nome": "nova", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "em_andamento", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "finalizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "aguardando", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "realizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)}
]

diretorio_base = r"\\algun diretorio"
diretorio_hoje = os.path.join(diretorio_base, hoje.strftime('%d-%m-%Y'))

if not os.path.exists(diretorio_hoje):
    os.makedirs(diretorio_hoje)

log_file_path = os.path.join(diretorio_hoje, "log.txt")
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')

for config in datas_config:
    situacao = config["nome"]
    data_inicial = config["data_inicial"]
    data_final = config["data_final"]
    
    pagina = 1
    nova_tabela = pd.DataFrame()
    
    while True:
        tempo_decorrido = time.time() - tempo_inicio
        
        if requisicoes_diferentes_consultas >= limite_diferentes_consultas:
            if tempo_decorrido < (tempo_limite_minutos * 60):
                tempo_espera = (tempo_limite_minutos * 60) - tempo_decorrido
                logging.info(f"Limite de requisições para consultas diferentes atingido. Esperando {tempo_espera:.1f} segundos.")
                time.sleep(tempo_espera)
            requisicoes_diferentes_consultas = 0
            tempo_inicio = time.time()

        try:
            resposta = coletar_dados(pagina, data_inicial, data_final, situacao)
            logging.info(f"Requisição feita: Página {pagina}, Situação {situacao}, URL {resposta.url}")
        except requests.RequestException as e:
            logging.error(f"Erro ao fazer a requisição: {e}")
            break

        if resposta.status_code == 200:
            logging.info(f"Requisição bem-sucedida: {resposta.url}")
        elif resposta.status_code == 429:
            logging.error(f"Erro 429: Limite de requisições excedido. Esperando 1 hora.")
            time.sleep(60 * 60)  
            continue 
        else:
            logging.error(f"Erro ao acessar a API: {resposta.status_code} - {resposta.text}")
            break
        
        try:
            informacoes = resposta.json()
        except ValueError as e:
            logging.error(f"Erro ao decodificar JSON: {e}")
            break

        if not informacoes:
            logging.info(f"Nenhum dado recebido na página {pagina} para a situação '{situacao}'.")
            break
        
        tabela = pd.DataFrame(informacoes)
        nova_tabela = pd.concat([nova_tabela, tabela], ignore_index=True)
        pagina += 1
        requisicoes_diferentes_consultas += 1

    excel_file = os.path.join(diretorio_hoje, f"dados_coletados_{situacao}.xlsx")
    nova_tabela.to_excel(excel_file, index=False)
    
    logging.info(f"Dados da situação '{situacao}' salvos em: {excel_file}")

print("Processo concluído.")


In [ ]:
import pandas as pd
import requests
import time
import os
from datetime import datetime, timedelta
import logging

def coletar_dados(pagina, data_inicial, data_final, situacao):
    url = "https://teste/rest/sdfdsdf/consultar/32443-34324343442-a683-dwdawdwdawdw"
    headers = {
        "chave-api": "dawdwaawd-28c2-343424-aa2b-dadwawwadwa"
    }
    params = {
        "pagina": pagina,
        "data_inicial": data_inicial.strftime('%d-%m-%Y'),
        "data_final": data_final.strftime('%d-%m-%Y'),
        "situacao": situacao
    }
    resposta = requests.get(url, headers=headers, params=params)
    return resposta

def concatenar_planilhas(diretorio):
    arquivos_excel = [f for f in os.listdir(diretorio) if f.endswith('.xlsx') and not f.startswith('planilhas_concatenadas')]
    planilhas = []
    
    for arquivo in arquivos_excel:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        df = pd.read_excel(caminho_arquivo)
        planilhas.append(df)
    
    if planilhas:
        planilha_concatenada = pd.concat(planilhas, ignore_index=True)
        arquivo_concatenado = os.path.join(diretorio, 'planilhas_concatenadas.xlsx')
        planilha_concatenada.to_excel(arquivo_concatenado, index=False)
        logging.info(f"Todas as planilhas foram concatenadas e salvas em: {arquivo_concatenado}")
    else:
        logging.info("Nenhuma planilha encontrada para concatenar.")


limite_diferentes_consultas = 10  
tempo_limite_minutos = 15  

requisicoes_diferentes_consultas = 0
tempo_inicio = time.time()

# Data atual
hoje = datetime.now().date()

# Definindo intervalos de datas (120 dias no total, 60 dias para trás e 60 dias para frente)
data_intervalo = 60
datas_config = [
    {"nome": "nova", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "em_andamento", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "finalizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "aguardando", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)},
    {"nome": "realizada", "data_inicial": hoje - timedelta(days=data_intervalo), "data_final": hoje + timedelta(days=data_intervalo)}
]

# Define o diretório base e cria o diretório para hoje
diretorio_base = r"\\direotriro\wewewq"
diretorio_hoje = os.path.join(diretorio_base, hoje.strftime('%d-%m-%Y'))

# Cria o diretório se não existir
if not os.path.exists(diretorio_hoje):
    os.makedirs(diretorio_hoje)

# Configuração do logging
log_file_path = os.path.join(diretorio_hoje, "log.txt")
logging.basicConfig(filename=log_file_path, level=logging.INFO, format='%(asctime)s %(levelname)s: %(message)s')


for config in datas_config:
    situacao = config["nome"]
    data_inicial = config["data_inicial"]
    data_final = config["data_final"]
    
    pagina = 1
    nova_tabela = pd.DataFrame()
    
    while True:
        tempo_decorrido = time.time() - tempo_inicio
        
        if requisicoes_diferentes_consultas >= limite_diferentes_consultas:
            if tempo_decorrido < (tempo_limite_minutos * 60):
                tempo_espera = (tempo_limite_minutos * 60) - tempo_decorrido
                logging.info(f"Limite de requisições para consultas diferentes atingido. Esperando {tempo_espera:.1f} segundos.")
                time.sleep(tempo_espera)
            requisicoes_diferentes_consultas = 0
            tempo_inicio = time.time()

        try:
            resposta = coletar_dados(pagina, data_inicial, data_final, situacao)
            logging.info(f"Requisição feita: Página {pagina}, Situação {situacao}, URL {resposta.url}")
        except requests.RequestException as e:
            logging.error(f"Erro ao fazer a requisição: {e}")
            break

        if resposta.status_code == 200:
            logging.info(f"Requisição bem-sucedida: {resposta.url}")
        elif resposta.status_code == 429:
            logging.error(f"Erro 429: Limite de requisições excedido. Esperando 1 hora.")
            time.sleep(61 * 61)  
            continue 
        else:
            logging.error(f"Erro ao acessar a API: {resposta.status_code} - {resposta.text}")
            break
        
        try:
            informacoes = resposta.json()
        except ValueError as e:
            logging.error(f"Erro ao decodificar JSON: {e}")
            break

        if not informacoes:
            logging.info(f"Nenhum dado recebido na página {pagina} para a situação '{situacao}'.")
            break
        
        tabela = pd.DataFrame(informacoes)
        nova_tabela = pd.concat([nova_tabela, tabela], ignore_index=True)
        pagina += 1
        requisicoes_diferentes_consultas += 1

    timestamp = datetime.now().strftime('%d-%m-%Y_%H-%M-%S')
    excel_file = os.path.join(diretorio_hoje, f"dados_coletados_{situacao}_{timestamp}.xlsx")
    nova_tabela.to_excel(excel_file, index=False)
    
    logging.info(f"Dados da situação '{situacao}' salvos em: {excel_file}")

concatenar_planilhas(diretorio_hoje)

print("Processo concluído.")
